## Auto-Programming

<div style="text-align:center">
    <img src="auto-programming1.png" height=400 width=250>
</div>

In [2]:
# We need an auto-programming script so we can have the option to choose
#whether we want our code to be alive and can decide for itself which 
#is the best way of write it's own code.

#### PROBLEMS AND DOUBTS

- What do we need? -> One-liner?, Script?, Alorythm? 
- How we change our own code with code?
- How to make the code understand itself?
- How to make not auto-destructive code?
- How can we supervise it?

#### Bayes' Theorem

This theorem provides a way to update the probability estimate of an event based on new evidence. 

$$
P(A|B) = \frac{P(A) \cdot P(B|A)}{P(B)}
$$

I think this can be useful for the future usage of the idea with the IA models or maybe not but here it is...

### First baby steps...

Self-modifying code involves programming techniques where the code actively alters its own instructions while it is running with Python:

```Here we have dynamic code execution and generation rather than modifying existing compiled code at runtime ... I think this it's going to teach us the path```

In [13]:
import random

code = """
def dynamic_function(x):
    return x * 2
"""
code2 = """
def dynamic_function(x):
    return x / 2
"""
codes = [code, code2]
#OPTION 1 exec_code = random.choice(codes)
#OPTION 2
index = int(random.random() * len(codes))
exec_code = codes[index]
# Dynamically define the function
exec(exec_code)

# Use the newly defined function
result = dynamic_function(5)
print("Result:", result)

Result: 10


### interesting blogs and a video

In [15]:
#like that: 
#https://wiki.c2.com/?SelfModifyingCode
#https://stackoverflow.com/questions/45175647/self-modifying-python-script
#https://wiki.c2.com/?ConsideredHarmful
#https://www.youtube.com/watch?v=SWU_DgjSwRU

Let's try modify the script file using ```__file__``` in python

> I'm going to use a separate script "self_modify.py" to test the code

### The code:
<pre>
def updateCount():
    print("Updating count...")

    # Open the file and read the code
    fin = open(__file__, 'r')
    code = fin.read()
    fin.close()
    # Split the code into lines
    lines = code.split('\n')

    # Process the second line to increment the count
    if lines[1].startswith('# Count: '):
        try:
            count = int(lines[1].split(': ')[1])
            lines[1] = f'# Count: {count + 1}'
            print(f"New count is {count + 1}")
        except ValueError:
            print("Error: The count on the second line is not a valid integer.")
            return
    else:
        print("Error: The second line is not formatted correctly.")
        return
    # Reassemble the modified code
    code = '\n'.join(lines)
    # Write the modified code back to the file
    fout = open(__file__, 'w')
    fout.write(code)
    fout.close()
# Call the function
updateCount()
</pre>

### Better code:
<pre>
import random
import time

# Define the two versions of the function
# START updateCountV1
def updateCountV1():
    print("Version 1 of the function is running.")
# END updateCountV1

# START updateCountV2
def updateCountV2():
    print("Version 2 of the function is running.")
# END updateCountV2

# Function to determine the currently active function
def get_active_function(lines):
    for line in lines:
        if 'def updateCountV1():' in line and not line.strip().startswith('#'):
            return updateCountV1
        elif 'def updateCountV2():' in line and not line.strip().startswith('#'):
            return updateCountV2
    return None

# Function to modify the script
def modify_script(active_function):
    with open(__file__, 'r') as file:
        lines = file.readlines()

    # Use random to select the new active function
    new_active_function = random.choice([updateCountV1, updateCountV2])

    in_function_block = False
    process_lines = False
    with open(__file__, 'w') as file:
        for line in lines:
            if 'START updateCountV1' in line or 'START updateCountV2' in line:
                in_function_block = True
                process_lines = new_active_function.__name__ in line
                file.write('#' + line.lstrip('#'))  # Always comment the START line
                continue

            if 'END updateCountV1' in line or 'END updateCountV2' in line:
                in_function_block = False
                file.write('#' + line.lstrip('#'))  # Always comment the END line
                continue

            if in_function_block and process_lines:
                file.write(line.lstrip('#'))  # Uncomment the lines for the active function
            elif in_function_block:
                file.write('#' + line.lstrip('#'))  # Comment the lines for the inactive function
            else:
                file.write(line)  # Write all other lines as is

    return new_active_function

# Read the current active function
with open(__file__, 'r') as file:
    current_lines = file.readlines()

current_active_function = get_active_function(current_lines)
if not current_active_function:
    current_active_function = updateCountV1  # Default to V1 if none is active

for i in range(2):
    # Modify the script and update the active function
    current_active_function = modify_script(current_active_function)

    # Call the active function
    current_active_function()
    time.sleep(2)

</pre>